# Course Scheduling System Demonstration

This notebook demonstrates the setup and utilization of a course scheduling system. It incorporates a custom scheduling model and a testing framework to assign teachers to courses based on their preferences and availability.



# Import Libraries

In [1]:
#import libraries

import pyomo.environ as pyo
import numpy as np
import pandas as pd


# Define the Course Scheduler Model

In [2]:
class course_scheduler:
    def __init__(self, days, hours, courses_overall, max_hours_per_day, teachers_Subject, preferencas_dias_professores, salas, discPreferenciasSala):
        self.days = days
        self.hours = hours
        self.courses_overall = courses_overall
        self.max_hours_per_day = max_hours_per_day
        self.teachers_Subject = dict(teachers_Subject)
        self.preferencas_dias_professores = dict(preferencas_dias_professores)
        self.salas = dict(salas)
        self.discPreferenciasSala = dict(discPreferenciasSala)
        self.models = []

    def create_model(self):
        for courses in self.courses_overall:
            model = pyo.ConcreteModel()
            # Sets
            model.days = pyo.Set(initialize=self.days)
            model.hours = pyo.Set(initialize=self.hours)
            model.courses = pyo.Set(initialize=courses.keys())

            # Parameters
            model.teacher_indices = pyo.Set(initialize=self.teachers_Subject.keys())
            model.teacher = pyo.Param(model.teacher_indices, initialize=self.teachers_Subject)
            
            # Consecutive blocks constraint
            model.consecutive_blocks_constraint = pyo.Constraint(model.days, model.hours, model.courses, rule=consecutive_blocks_rule)

            model.rooms = pyo.Set(initialize=self.salas.keys())

            # Parameters
            model.hours_per_course = pyo.Param(model.courses, initialize=courses)
            model.max_hours_per_day = self.max_hours_per_day
            # Max hours per week per course
            model.max_hours_per_week = {course: hours for course, hours in courses.items()}

            # Variables
            model.schedule = pyo.Var(model.days, model.hours, model.courses, domain=pyo.Binary)
            model.room_assignment = pyo.Var(model.days, model.hours, model.courses, model.rooms, domain=pyo.Binary)

           

            def max_hours_per_week_rule(model, c):
                return sum(model.schedule[d, h, c] for d in model.days for h in model.hours) <= model.max_hours_per_week[c]

            def max_hours_per_day_per_course_rule(model, d, c):
                return sum(model.schedule[d, h, c] for h in model.hours) <= 2  # Limit each subject to 2 hours per day


            def room_availability_rule(model, d, h, c):
                return sum(model.room_assignment[d, h, c, room] for room in model.rooms) == 1

            def room_preferences_rule(model, d, h, c):
                course_preferences = self.discPreferenciasSala.get(c, [])
                if course_preferences:
                    return sum(model.room_assignment[d, h, c, room] for room in course_preferences) >= model.schedule[d, h, c]
                else:
                    return pyo.Constraint.Skip  # Skip constraint if there are no preferences

            def consecutive_blocks_rule(model, d, h, c):
                if h == model.hours.first() and h != model.hours.last():
                    # If the first hour of the day is scheduled, the next hour must also be scheduled
                    return model.schedule[d, h, c] <= model.schedule[d, model.hours.next(h), c]
                elif h == model.hours.last() and h != model.hours.first():
                    # If the last hour of the day is scheduled, the previous hour must also be scheduled
                    return model.schedule[d, h, c] <= model.schedule[d, model.hours.prev(h), c]
                elif h != model.hours.first() and h != model.hours.last():
                    # For any other hour, if the class is scheduled, either the previous or next hour must also be scheduled
                    return model.schedule[d, h, c] <= (model.schedule[d, model.hours.prev(h), c] + model.schedule[d, model.hours.next(h), c])
                else:
                    # If there's only one hour in the set, then we don't enforce consecutive blocks
                    return pyo.Constraint.Skip

                # Teacher preferences constraint
            # Define constraint to enforce teacher availability
            def teacher_availability_constraint_rule(model, d, h, c):
                assigned_teacher = model.teacher[c]  # Get the assigned teacher for the course
                if assigned_teacher in self.preferencas_dias_professores:
                    # If the assigned teacher is in the preferences dictionary
                    if d in self.preferencas_dias_professores[assigned_teacher]:
                        # If the assigned teacher is available on the scheduled day, return True
                        return model.schedule[d, h, c] <= 1
                    else:
                        # If the assigned teacher is not available on the scheduled day, return False
                        return model.schedule[d, h, c] <= 0
                else:
                    # If the assigned teacher is not in the preferences dictionary, return True
                    return model.schedule[d, h, c] <= 1

            # Add the constraint to the model
            model.teacher_availability_constraint = pyo.Constraint(model.days, model.hours, model.courses, rule=teacher_availability_constraint_rule)

            model.room_availability_constraint = pyo.Constraint(model.days, model.hours, model.courses, rule=room_availability_rule)
            model.room_preferences_constraint = pyo.Constraint(model.days, model.hours, model.courses, rule=room_preferences_rule)

            model.consecutive_blocks_constraint = pyo.Constraint(model.days, model.hours, model.courses, rule=consecutive_blocks_rule)

            model.max_hours_per_day_per_course_constraint = pyo.Constraint(model.days, model.courses, rule=max_hours_per_day_per_course_rule)
            model.max_hours_per_week_constraint = pyo.Constraint(model.courses, rule=max_hours_per_week_rule)


            # Objective
            def objective_rule(model):
                return sum(model.schedule[d, h, c] * model.hours_per_course[c] for d in model.days for h in model.hours for c in model.courses)

            model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

            self.models.append(model)


    def solve(self):
        for model in self.models:
            solver = pyo.SolverFactory('cbc')
            result = solver.solve(model)
            print(result)
    

    def _create_course_abbreviations(self):
        # Abbreviations for MIAA courses
        self.abbreviations_miaa = {
            'Computational Tools for Data Science': 'CTDS',
            'Mathematical Foundations for Artificial Intelligence': 'MFAI',
            'Fundamentals of Artificial Intelligence': 'FAI',
            'Statistical Models for AI': 'SMAI',
            'Machine Learning Algorithms': 'MLA',
        }
        # Abbreviations for LEEC courses
        self.abbreviations_leec = {
            'Cálculo': 'CAL',
            'Matemática Discreta e Álgebra Linear': 'MDAL',
            'Teoria dos Circuitos Elétricos': 'TCE',
            'Sistemas Digitais': 'SD',
            'Programação Imperativa': 'PI',
        }

    # Ensure the following method is correctly aligned with the class definition.
    ############################################################################################################
    def print_schedule(self):
        # Create course abbreviations
        self._create_course_abbreviations()

        for idx, model in enumerate(self.models):
            print(f"Schedule for Class {idx+1}:")

            # Set uniform column width for all columns including Time/Day
            column_width = 16  # Adjust the width as needed for your content

            # Calculate the total width of the table based on the uniform column width
            num_columns = len(model.days) + 1  # Number of days + Time/Day column
            total_width = num_columns * (column_width + 3) - 1  # Including separators

            # Print the top border of the table
            print("+" + "-" * (total_width) + "+")

            # Print header row with uniform column width
            header = "| {:^{}} |".format("Time/Day", column_width)
            for day in model.days:
                header += " {:^{}} |".format(day, column_width)
            print(header)
            print("+" + "-" * (total_width) + "+")

            # Print schedule rows with uniform column width
            for hour in model.hours:
                row = "| {:^{}} |".format(hour, column_width)
                for day in model.days:
                    cell_content = "No class"
                    room_assigned = ""
                    for course in model.courses:
                        if model.schedule[day, hour, course].value == 1:
                            course_abbr = self.abbreviations_miaa.get(course, self.abbreviations_leec.get(course, course))
                            for room in model.rooms:
                                if model.room_assignment[day, hour, course, room].value == 1:
                                    room_assigned = room
                                    break
                            cell_content = f"{course_abbr} ({room_assigned})"
                            break
                    # Add cell content with the same column width
                    row += " {:^{}} |".format(cell_content, column_width)
                print(row)
                print("+" + "-" * (total_width) + "+")
        print()






# Setting Up the Scheduler Parameters

In [3]:
from scheduler_model import course_scheduler
import random



def assign_teachers_to_courses(teachers_Subject, preferencas_dias_professores):
    """
    Assigns one respective teacher for each course based on teacher availability preferences.
    """
    course_teachers = {}
    for course, teachers in teachers_Subject.items():
        available_teachers = [teacher for teacher in teachers if preferencas_dias_professores.get(teacher)]
        if available_teachers:
            selected_teacher = random.choice(available_teachers)
            course_teachers[course] = selected_teacher
        else:
            # If no available teacher found, assign None
            course_teachers[course] = None
    return course_teachers




if __name__ == "__main__":
    #days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    #hours = [f"Hour_{i}" for i in range(8, 18)]
    # Adjusting hours to reflect actual time slots for readability
    #hours = ["9:00 AM", "10:00 AM", "11:00 AM", "12:00 PM", "1:00 PM", "2:00 PM", "3:00 PM", "4:00 PM", "5:00 PM", "6:00 PM"]
    # Define days, hours, and course units for both programs (simplified here)
    
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    hours = ["09:00-10:00", "10:00-11:00", "11:00-12:00", "12:00-13:00", "13:00-14:00", "14:00-15:00", "15:00-16:00", "16:00-17:00"]
    course_units_miaa = {'CTDS': 3, 'MFAI': 3, 'FAI': 4, 'SMAI': 2, 'MLA': 4}
    course_units_leec = {'CAL': 4, 'MDAL': 3, 'TCE': 3, 'SD': 3, 'PI': 4}
    max_hours_per_day = 6


    salas = {'salaA': 0,'salaB': 0, 'salaC': 0,'salaD': 0,'salaE': 0,'ginasio': 0,'lab': 0,'salaArtes': 0}

    discPreferenciasSala = {"Computational Tools for Data Science" : ["salaA", "salaB"] , 
                        'Mathematical Foundations for Artificial Intelligence': [],
                        'Fundamentals of Artificial Intelligence': [],
                        'Statistical Models for AI': [],
                        'Machine Learning Algorithms': ["salaE"],
                        'Cálculo': ["salaD", "salaC", "salaD"],
                        'Matemática Discreta e Álgebra Linear': ["salaB"],
                        'Teoria dos Circuitos Elétricos': [],
                        'Sistemas Digitais': [],
                        'Programação Imperativa': ["salaA", "salaD"],
        }

    teachers_Subject = {"Computational Tools for Data Science" : ["Celia Oliveira", "Natalia Costa", "Carla Ferreira"] , 
                        'Mathematical Foundations for Artificial Intelligence': ["Celia Oliveira", "Beatriz Rodrigues", "Ana Oliveira"],
                        'Fundamentals of Artificial Intelligence': ["Clara Pereira", "Matilde Carvalho", "Ana Oliveira"],
                        'Statistical Models for AI': ["Carla Sousa", "Celia Santos"],
                        'Machine Learning Algorithms': ["Celia Oliveira", "Natalia Costa", "Carla Ferreira"],
                        'Cálculo': ["Silvia Fernandes", "Matilde Silva"],
                        'Matemática Discreta e Álgebra Linear': ["Carla Martins"],
                        'Teoria dos Circuitos Elétricos': ["Carla Sousa", "Maria Rodrigues", "Matilde Carvalho"],
                        'Sistemas Digitais': ["Silvia Fernandes", "Clara Oliveira", "Carla Ferreira"],
                        'Programação Imperativa': ["Clara Varzim", "Carla Ferreira"],
        }
    

    preferencas_dias_professores = {
                        "Celia Oliveira" : ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] , 
                        'Natalia Costa': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] ,  
                        'Carla Ferreira': ['Wednesday', 'Thursday', 'Friday'] , 
                        'Beatriz Rodrigues': ['Monday', 'Tuesday'] , 
                        'Ana Oliveira': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] , 
                        'Clara Pereira': ['Monday', 'Tuesday', 'Wednesday', 'Thursday'] , 
                        'Matilde Carvalho': ['Monday', 'Tuesday', 'Wednesday'] ,  
                        'Carla Sousa': ['Wednesday', 'Thursday', 'Friday'] ,  
                        'Celia Santos': ['Tuesday', 'Wednesday', 'Thursday', 'Friday'] , 
                        'Silvia Fernandes': ['Monday', 'Tuesday', 'Thursday', 'Friday'] ,  
                        'Matilde Silva': ['Monday', 'Tuesday', 'Thursday', 'Friday'] ,  
                        'Marisa Oliveira':['Wednesday', 'Thursday', 'Friday'] , 
                        'Bruna Martins': ['Monday', 'Tuesday', 'Wednesday'] , 
                        'Carla Martins':['Wednesday'] ,  
                        'Maria Rodrigues': ['Monday', 'Tuesday', 'Wednesday', 'Friday'] , 
                        'Matilde Carvalho':['Monday', 'Tuesday', 'Friday'] ,  
                        'Clara Oliveira': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] ,  
                        'Clara Varzim': ['Tuesday', 'Wednesday', 'Thursday'] , 
    }

    professoresAvaiability = {
                        "Celia Oliveira" : 0 , 
                        'Natalia Costa': 0 ,  
                        'Carla Ferreira': 0, 
                        'Beatriz Rodrigues': 0,
                        'Ana Oliveira': 0,
                        'Clara Pereira': 0, 
                        'Matilde Carvalho': 0, 
                        'Carla Sousa': 0,  
                        'Celia Santos': 0, 
                        'Silvia Fernandes': 0,  
                        'Matilde Silva': 0, 
                        'Marisa Oliveira':0,
                        'Bruna Martins': 0,
                        'Carla Martins':0,
                        'Maria Rodrigues':0,
                        'Matilde Carvalho':0,
                        'Clara Oliveira': 0,
                        'Clara Varzim': 0,
    }


    course_units_miaa = {
        'Computational Tools for Data Science': 3,
        'Mathematical Foundations for Artificial Intelligence': 2,
        'Fundamentals of Artificial Intelligence': 3,
        'Statistical Models for AI': 2,
        'Machine Learning Algorithms': 2,
    }

    course_units_leec = {
        'Cálculo': 4,
        'Matemática Discreta e Álgebra Linear': 4 ,
        'Teoria dos Circuitos Elétricos': 3,
        'Sistemas Digitais': 3,
        'Programação Imperativa': 4,
    }

    courses_overall = [course_units_miaa , course_units_leec]

    max_hours_per_day = 8

    teachers_chosen = {}

    # Use the new function to assign teachers to courses
    assigned_teachers = assign_teachers_to_courses(teachers_Subject, preferencas_dias_professores)
    print("Assigned Teachers to Courses:")
    for course, teacher in assigned_teachers.items():
        print(f"{course}: {teacher}")
        teachers_chosen[course] = teacher

    # Assume course_units_miaa, course_units_leec, days, hours, and max_hours_per_day are already defined
    scheduler = course_scheduler(days, hours, courses_overall, max_hours_per_day,teachers_chosen, preferencas_dias_professores,salas,discPreferenciasSala)

    #scheduler = course_scheduler(days, hours, course_units_miaa, course_units_leec, max_hours_per_day)
    scheduler.create_model()
    result = scheduler.solve()
    scheduler.print_schedule()  # Add this line to print the schedule


Assigned Teachers to Courses:
Computational Tools for Data Science: Natalia Costa
Mathematical Foundations for Artificial Intelligence: Ana Oliveira
Fundamentals of Artificial Intelligence: Matilde Carvalho
Statistical Models for AI: Carla Sousa
Machine Learning Algorithms: Celia Oliveira
Cálculo: Matilde Silva
Matemática Discreta e Álgebra Linear: Carla Martins
Teoria dos Circuitos Elétricos: Maria Rodrigues
Sistemas Digitais: Clara Oliveira
Programação Imperativa: Carla Ferreira
'Any'. The default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend the
domain of this Paramto be 'Any', you can suppress this warning by explicitly
specifying 'within=Any' to the Param constructor. (deprecated in 5.6.9, will
be removed in (or after) 6.0) (called from
C:\Users\uliss\AppData\Roaming\Python\Python311\site-
packages\pyomo\core\base\indexed_component.py:716)

Problem: 
- Name: unknown
  Lower bound: 30.0
  Upper bound: 3